In [1]:
import pandas as pd
import os

In [2]:
os.getcwd()

'/home/apeters/Documents/Research/ShippingEmissionsProject/BulkShipDev/src'

In [3]:
gpd = (
    # pd.read_csv("./data/potportcalls_speed_EU.csv")
    pd.read_csv("../../SharedData/Bulk/potportcalls_speed_EU_Localtest.csv")
    # .drop(columns=["geometry"])
)
len(gpd)
print(gpd.head())

       imo  pot_trip  latitude  longitude ISO_3digit   EU
0 -7926162        59  9.033334 -80.695000        PAN  0.0
1 -7926162        67  9.036667 -80.688332        PAN  0.0
2 -7926162        79  9.031833 -80.688667        PAN  0.0
3 -7926162       164  9.026667 -80.713333        PAN  0.0
4 -7926162       173  9.027273 -80.717422        PAN  0.0


In [4]:
gpd_CC = (
    pd.read_csv("./data/potportcalls_speed_EU.csv")
    # .drop(columns=["geometry"])
)
len(gpd_CC)
print(gpd_CC.head())
gpd_imos = gpd_CC["imo"].drop_duplicates()
gpd_imos

       imo  pot_trip   latitude  longitude ISO_3digit   EU
0 -9268942        17 -52.758640 -74.164162        CHL  0.0
1 -9205615        75 -49.155926 -74.366585        CHL  0.0
2 -9496941        98 -49.151718 -74.390610        CHL  0.0
3 -9241504        31 -48.764244 -74.362297        CHL  0.0
4 -9754094        30 -48.753441 -74.357834        CHL  0.0


0        -9268942
1        -9205615
2        -9496941
3        -9241504
4        -9754094
           ...   
823781   -9036430
824896   -9363687
829414    9727053
831870   -9708643
833433    9323871
Name: imo, Length: 14161, dtype: int64

In [ ]:
(
    gpd
    .merge(
        gpd_CC,
        how="outer",
        on=["imo", "pot_trip"],
        indicator=True,
        suffixes=("_gpd", "_gpd_CC")
    )
    .value_counts("_merge")
)

_merge
right_only    841963
both            1227
left_only          0
Name: count, dtype: int64

In [6]:
qgis = (
    pd.read_csv("./data/potportcalls_speed_EU_qgis.csv", low_memory=False)
    .drop(columns=["fid", "id"])
    .query("imo.isin(@gpd_imos)")
)
len(qgis)
qgis.head()

,imo,pot_trip,latitude,longitude,ISO_3digit,EU
0,-9179696,133,22.781086,116.097900,CHN,0
1,-9221190,16,22.817719,116.173347,CHN,0
2,9159555,99,22.839308,116.102600,CHN,0
3,-9208502,62,22.889664,116.408730,CHN,0
4,-9226671,174,22.753071,115.805969,CHN,0


In [7]:
merged = (
    gpd_CC
    .merge(
        qgis,
        how="outer",
        on=["imo", "pot_trip"],
        indicator=True,
        suffixes=("_gpd", "_qgis")
    )
)

merged["lat"] = merged["latitude_gpd"].combine_first(merged["latitude_qgis"])
merged["lon"] = merged["longitude_gpd"].combine_first(merged["longitude_qgis"])
merged["_merge"] = merged["_merge"].replace({
    "left_only": "gpd_only",
    "right_only": "qgis_only"
})
merged.to_csv("./data/compare_qgis_gpd_portcalls.csv", index=False)

/tmp/ipykernel_2203405/952623221.py:14: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  merged["_merge"] = merged["_merge"].replace({


In [8]:
merged.value_counts("_merge")

_merge
both         838146
qgis_only     14110
gpd_only       5044
Name: count, dtype: int64

In [9]:
merged.query("_merge == 'both'")

,imo,pot_trip,latitude_gpd,longitude_gpd,ISO_3digit_gpd,EU_gpd,latitude_qgis,longitude_qgis,ISO_3digit_qgis,EU_qgis,_merge,lat,lon
0,-9957749,4,10.361451,107.242905,VNM,0.0,10.361451,107.242905,VNM,0.0,both,10.361451,107.242905
1,-9957749,5,10.394067,107.016716,VNM,0.0,10.394067,107.016716,VNM,0.0,both,10.394067,107.016716
2,-9957749,6,10.329042,107.051369,VNM,0.0,10.329042,107.051369,VNM,0.0,both,10.329042,107.051369
3,-9957749,8,10.436783,106.967331,VNM,0.0,10.436783,106.967331,VNM,0.0,both,10.436783,106.967331
5,-9957749,10,10.349178,107.049088,VNM,0.0,10.349178,107.049088,VNM,0.0,both,10.349178,107.049088
...,...,...,...,...,...,...,...,...,...,...,...,...,...
857295,9917490,1,31.715370,121.110840,CHN,0.0,31.715370,121.110840,CHN,0.0,both,31.715370,121.110840
857296,9917490,3,31.720469,121.110031,CHN,0.0,31.720469,121.110031,CHN,0.0,both,31.720469,121.110031
857297,9917490,9,-23.969328,-46.289677,BRA,0.0,-23.969328,-46.289677,BRA,0.0,both,-23.969328,-46.289677
857298,9917490,21,30.433411,49.057728,IRN,0.0,30.433411,49.057728,IRN,0.0,both,30.433411,49.057728


In [10]:
merged.query("_merge == 'gpd_only'").head(30)

,imo,pot_trip,latitude_gpd,longitude_gpd,ISO_3digit_gpd,EU_gpd,latitude_qgis,longitude_qgis,ISO_3digit_qgis,EU_qgis,_merge,lat,lon
291,-9914371,3,36.117825,129.481506,KOR,0.0,NaN,NaN,NaN,NaN,gpd_only,36.117825,129.481506
478,-9912074,7,34.371651,133.434998,JPN,0.0,NaN,NaN,NaN,NaN,gpd_only,34.371651,133.434998
605,-9911587,29,56.133476,10.284895,DNK,1.0,NaN,NaN,NaN,NaN,gpd_only,56.133476,10.284895
1310,-9907847,16,67.548988,-164.189957,USA,0.0,NaN,NaN,NaN,NaN,gpd_only,67.548988,-164.189957
1365,-9906477,4,18.584999,-92.768333,MEX,0.0,NaN,NaN,NaN,NaN,gpd_only,18.584999,-92.768333
2126,-9899399,33,36.104618,129.482590,KOR,0.0,NaN,NaN,NaN,NaN,gpd_only,36.104618,129.482590
2154,-9898759,19,-2.344807,-44.309643,BRA,0.0,NaN,NaN,NaN,NaN,gpd_only,-2.344807,-44.309643
3176,-9892688,14,41.071445,1.298319,ESP,1.0,NaN,NaN,NaN,NaN,gpd_only,41.071445,1.298319
3264,-9891907,30,15.839382,-88.618149,GTM,0.0,NaN,NaN,NaN,NaN,gpd_only,15.839382,-88.618149
3312,-9891866,5,42.762512,132.968796,RUS,0.0,NaN,NaN,NaN,NaN,gpd_only,42.762512,132.968796


In [11]:
merged.query("_merge == 'qgis_only'").head(30)

,imo,pot_trip,latitude_gpd,longitude_gpd,ISO_3digit_gpd,EU_gpd,latitude_qgis,longitude_qgis,ISO_3digit_qgis,EU_qgis,_merge,lat,lon
4,-9957749,9,NaN,NaN,NaN,NaN,10.277464,107.068634,VNM,0.0,qgis_only,10.277464,107.068634
18,-9957749,31,NaN,NaN,NaN,NaN,10.277289,107.080490,VNM,0.0,qgis_only,10.277289,107.080490
21,-9957749,34,NaN,NaN,NaN,NaN,10.277264,107.067802,VNM,0.0,qgis_only,10.277264,107.067802
46,-9932103,5,NaN,NaN,NaN,NaN,22.148020,114.182205,CHN,0.0,qgis_only,22.148020,114.182205
113,-9922005,5,NaN,NaN,NaN,NaN,1.274633,103.942726,SGP,0.0,qgis_only,1.274633,103.942726
170,-9920057,4,NaN,NaN,NaN,NaN,33.690170,132.534470,JPN,0.0,qgis_only,33.690170,132.534470
261,-9916290,4,NaN,NaN,NaN,NaN,1.283277,103.968475,SGP,0.0,qgis_only,1.283277,103.968475
336,-9913688,8,NaN,NaN,NaN,NaN,20.860302,107.116135,VNM,0.0,qgis_only,20.860302,107.116135
337,-9913688,9,NaN,NaN,NaN,NaN,1.290002,103.991272,SGP,0.0,qgis_only,1.290002,103.991272
377,-9913652,9,NaN,NaN,NaN,NaN,2.795874,101.307495,MYS,0.0,qgis_only,2.795874,101.307495


In [12]:
qgis.query("abs(imo) == 9084229 and pot_trip == 51")

,imo,pot_trip,latitude,longitude,ISO_3digit,EU
302296,-9084229,51,59.694153,28.426901,RUS,0


In [14]:
gpd_CC.query("abs(imo) == 9084229 and pot_trip == 51")

,imo,pot_trip,latitude,longitude,ISO_3digit,EU
403226,-9084229,51,59.694153,28.426901,RUS,0.0
